### IMPORTING REQUIRED LIBRARIES AND MODULES :- 

In [58]:
import tensorflow as tf
import numpy as np

import math
import csv
from tensorflow.keras.models import Sequential as seq
from tensorflow.keras.layers import Dense as den
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from datetime import datetime
tf.random.set_seed(42)

### CREATING DATASET(GENERATING ANGLES)

In [59]:
'''
a1=[]
b1=[]
x11=-0.25

p = -1
for i in range(1440):
  x11 =x11+0.25
  y11=-0.25
  for j in range (720):
    y11 =y11+0.25
    a1.append(x11)
    b1.append(y11)

x =len(a1)
s = np.zeros((x,2), dtype = np.float32)
for i in range(x):
  p = p +1
  
  s[i][0] = a1[p]
  s[i][1] = b1[p]
  


fields = [ 'THETA1' ,'Theta2']

filename = '/content/angles_larger_data.csv'
with open(filename, 'w') as csvfile: 
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(fields)
  csvwriter.writerows(s)
'''

"\na1=[]\nb1=[]\nx11=-0.25\n\np = -1\nfor i in range(1440):\n  x11 =x11+0.25\n  y11=-0.25\n  for j in range (720):\n    y11 =y11+0.25\n    a1.append(x11)\n    b1.append(y11)\n\nx =len(a1)\ns = np.zeros((x,2), dtype = np.float32)\nfor i in range(x):\n  p = p +1\n  \n  s[i][0] = a1[p]\n  s[i][1] = b1[p]\n  \n\n\nfields = [ 'THETA1' ,'Theta2']\n\nfilename = '/content/angles_larger_data.csv'\nwith open(filename, 'w') as csvfile: \n  csvwriter = csv.writer(csvfile)\n  csvwriter.writerow(fields)\n  csvwriter.writerows(s)\n"

In [60]:
angle = np.loadtxt('angles_larger_data.csv', delimiter=',', dtype=np.float32 , skiprows=1)

In [61]:
a =math.pi/180

def x_data(theta_1,theta_2):
  theta1 = a*theta_1
  theta2 = a*theta_2
  x =  math.cos(theta1) + math.cos(theta1 + theta2)
  return float(x)

def y_data(theta_1,theta_2):
  theta1 = a*theta_1
  theta2 = a*theta_2
  y = math.sin(theta1) + math.sin(theta1 + theta2)
  return float(y)

x_1 = np.zeros(1036800)
y_1 =np.zeros(1036800)

s = np.zeros((1036800,5), dtype = np.float32)

for k in range(1036800):
  
    x_1[k] = x_data(angle[k][0],angle[k][1])
    
    y_1[k] = y_data(angle[k][0],angle[k][1])

p = -1
for i in range(1036800):
  p = p +1
  s[i][0] = p
  s[i][1] = x_1[p]
  s[i][2] = y_1[p]
  s[i][3] = angle[p][0]
  s[i][4] = angle[p][1]


In [62]:
s =tf.constant(s)
data1 = s
train_input = data1[:836800,1:3]
train_output1 = data1[:836800 ,3:4 ]
train_output2 = data1[:836800 ,4: ]
val_input = data1[836800:936800,1:3]
val_output1 = data1[836800:936800 ,3:4 ]
val_output2 = data1[836800:936800 ,4: ]
test_input = data1[936800:,1:3]
test_output1 = data1[936800: ,3:4 ]
test_output2 = data1[936800: ,4: ]
test_input_short = data1[836870:836880,1:3]
test_output_short1 = data1[836870:836880, 3:4]
test_output_short2 = data1[836870:836880, 4:]

### Defining Neural Net for theta1

In [87]:
model1 = seq([den(2 ,activation='relu'),
              den(16,activation ='relu'),
              den(32 ,activation ='relu'),
              den(48 ,activation ='relu'),
              den(32 ,activation ='relu'),
              den(16 ,activation ='relu'),
              den(1,activation ='relu')])
model1.compile(optimizer ='adam', loss ='mae')

### Training and validatating  model1

In [88]:
model1.fit(train_input, train_output1,
          validation_data=(val_input,val_output1) ,epochs= 250 )

Epoch 1/250
26150/26150 [==============================] - 10s 388us/step - loss: 25.6738 - val_loss: 13.3704
Epoch 2/250
26150/26150 [==============================] - 11s 420us/step - loss: 11.7289 - val_loss: 9.4775
Epoch 3/250
26150/26150 [==============================] - 11s 417us/step - loss: 8.4380 - val_loss: 6.9879
Epoch 4/250
26150/26150 [==============================] - 10s 390us/step - loss: 6.9336 - val_loss: 7.0981
Epoch 5/250
26150/26150 [==============================] - 10s 387us/step - loss: 6.5053 - val_loss: 5.7972
Epoch 6/250
26150/26150 [==============================] - 10s 400us/step - loss: 6.0892 - val_loss: 5.0476
Epoch 7/250
26150/26150 [==============================] - 10s 392us/step - loss: 5.9213 - val_loss: 5.6134
Epoch 8/250
26150/26150 [==============================] - 10s 386us/step - loss: 5.7091 - val_loss: 5.6184
Epoch 9/250
26150/26150 [==============================] - 10s 386us/step - loss: 5.4896 - val_loss: 4.4825
Epoch 10/250
26150/26150 

#### Evaluating the performance of model1

In [89]:
model1.evaluate(test_input, test_output1)

3125/3125 [==============================] - 1s 318us/step - loss: 3.5363


3.536292552947998

#### predicting and comparing with a small data

In [75]:
test = model1.predict(test_input_short)
print(test)
print(test_output_short1)


[[174.65665 ]
 [ 59.559933]
 [166.38606 ]
 [305.62775 ]
 [209.93216 ]
 [352.89166 ]
 [ 62.80483 ]
 [166.91739 ]
 [222.78976 ]
 [ 86.70197 ]]
tf.Tensor(
[[174.5 ]
 [ 60.75]
 [166.  ]
 [308.  ]
 [209.5 ]
 [359.75]
 [ 73.25]
 [166.5 ]
 [223.  ]
 [ 88.  ]], shape=(10, 1), dtype=float32)


### Training and validatating  model2

In [76]:
model2 = seq([den(2 ,activation='relu'),
              den(16,activation ='relu'),
              den(32 ,activation ='relu'),
              den(32 ,activation ='relu'),
              den(32 ,activation ='relu'),
              den(16 ,activation ='relu'),
              den(1,activation ='relu')])
model2.compile(optimizer ='adam', loss ='mae')

#### Training and validating model2

In [53]:
model2.fit(train_input, train_output2,
          validation_data=(val_input,val_output2) ,epochs= 120)

Epoch 1/120
26150/26150 [==============================] - 11s 414us/step - loss: 30.7529 - val_loss: 30.2053
Epoch 2/120
26150/26150 [==============================] - 10s 380us/step - loss: 7.4089 - val_loss: 1.4738
Epoch 3/120
26150/26150 [==============================] - 10s 378us/step - loss: 1.7643 - val_loss: 1.2030
Epoch 4/120
26150/26150 [==============================] - 10s 379us/step - loss: 1.5064 - val_loss: 1.4539
Epoch 5/120
26150/26150 [==============================] - 10s 380us/step - loss: 1.3871 - val_loss: 1.0883
Epoch 6/120
26150/26150 [==============================] - 10s 379us/step - loss: 1.3138 - val_loss: 1.5101
Epoch 7/120
26150/26150 [==============================] - 10s 377us/step - loss: 1.2572 - val_loss: 0.9605
Epoch 8/120
26150/26150 [==============================] - 10s 381us/step - loss: 1.2150 - val_loss: 1.2971
Epoch 9/120
26150/26150 [==============================] - 10s 378us/step - loss: 1.1819 - val_loss: 1.3228
Epoch 10/120
26150/26150 [

#### evaluating the model2

In [54]:
model2.evaluate(test_input, test_output2)

3125/3125 [==============================] - 1s 282us/step - loss: 0.5109


0.5109319686889648

#### predicting the outcomes of model2 over a small data


In [55]:
test = model2.predict(test_input_short)
print(test)
print(test_output_short2)

[[ 21.16051 ]
 [ 97.69935 ]
 [143.62082 ]
 [159.78867 ]
 [147.71585 ]
 [146.0476  ]
 [170.47035 ]
 [135.7854  ]
 [ 55.195213]
 [137.12183 ]]
tf.Tensor(
[[ 21.25]
 [ 98.25]
 [143.5 ]
 [160.25]
 [148.  ]
 [146.5 ]
 [170.75]
 [136.  ]
 [ 55.25]
 [137.25]], shape=(10, 1), dtype=float32)



### saving the models

In [158]:
model1.save('parameters_model1.h5')
model2.save('parameters_model2.h5')